# RA

# Índice

[Enunciado](#Enunciado)

[Solución](#Solución)

[Tardanza del código](#Tardanza-del-código)

[Objetos disponibles](#Objetos-disponibles)

[Prueba](#Prueba)

[Teoría utilizada (bibliografía)](#Teoría-utilizada)

## Enunciado

Crea un efecto de realidad aumentada en el que el usuario desplace objetos virtuales hacia posiciones marcadas con el ratón.

## Solución

En primer lugar, se van a explicar las variables importantes para el funcionamiento del código:

1. **desplazamiento**: Contiene el punto 3D destino del objeto virtual 
2. **actual**: Contiene el punto 3D en el que se encuentra el objeto virtual
3. **start** y **now**: start contiene la última vez que se mostró un frame y now contiene el tiempo actual. Con start y now se puede saber cuanto tiempo ha transcurrido desde la última vez que se mostró un frame (y por lo tanto, desde que se movió el objeto).
4. **K_Matriz**: Matriz de calibración de la cámara. Si se usa otra habría que cambiarla.
5. **marker**: Los puntos 3D de los que se compone el marcador.
6. **objeto**: Los puntos 3D de los que se compone el objeto virtual
7. **point** y **pulsado**: point contiene el pixel que se pulsa en la pantalla, y pulsado contiene True si se acaba de pulsar y aún no se ha calculado a dónde debe ir el objeto virtual (*desplazamiento*), y False si ya se ha calculado 

Hay otras variables, pero no son globales, sino que se crean en cada frame. Por ello, se prosigue con la explicación del código del bucle principal (se llama una vez por frame):

1. Se guarda el tiempo actual en **now**.

2. Se extraen los contornos del frame (extractContours de umucv) y se almacenan los que se pueden reducir a 6 vértices (el marcador tiene 6 vértices y queremos encontrar el marcador).

    2.1. Si no ha encontrado ninguno, se muestra el frame y se pasa a otra iteración del bucle principal.

3. Se usa la función *bestPose* explicada en [los apuntes de la asignatura](https://github.com/albertoruiz/umucv/blob/96a0e8bcc9e95151c309ac48f743e4fd8a90a77c/notebooks/camera.ipynb) para iterar por todos los contornos de 6 vértices y obtener el que más probable sea el marcador. 

    bestPose dado el modelo y el contorno, calcula la matriz de cámara que debería haber para que el marcador con esa matriz de cámara diera el contorno. Devuelve la matriz de cámara y el error de reproyección.
    
    De este modo, si se tiene mucho error, indica que probablemente no sea el marcador lo que se ha detectado. Por ello, nos quedamos con el contorno que menor error de reproyección tenga (y por lo tanto, el que más probablemente sea el marcador). Almacenamos la matriz de cámara de la mejor opción, los puntos que constituyen su contorno, y su error de reproyección.

    3.1. Si el error no es menor a 4, se considera que no se ha encontrado el marcador en el frame, por lo que se muestra el frame y se pasa a otra iteración del bucle principal.

4. Se dibuja el contorno del marcador encontrado en rojo.

    4.1. Si no se había marcado ningún punto, se dibuja en verde el objeto virtual encima del marcador. Para ello, se obtienen los píxeles correspondientes a los puntos del objeto virtual aplicándole la matriz de cámara (htrans) al objeto en la posición actual. El objeto en la posición actual (que en el inicio es la posición en la que se encuentra el marcador) se obtiene de aplicar un desplazamiento al objeto (htrans con la función *desp* de la posición actual).

5. Si *pulsado* es True, se debe calcular el desplazamiento al que debe ir el objeto virtual. Para ello, se elimina la tercera columna de la matriz de cámara (es la columna que se refiere al eje z, si se elimina entonces se obtiene una altura de 0), y se invierte. Una vez invertida, se aplica al punto marcado (*point*). De esto se obtiene la posición del punto en 3D (*desplazamiento*) ya que, como la matriz de cámara funciona para obtener de un punto en 3D al píxel en el frame, si se invierte se puede calcular del píxel en el frame al punto en 3D. *pulsado* se pone a falso para que no se vuelva a calcular.

    Es importante que se calcule la posición, a partir del píxel donde debe ir el objeto, enseguida y después no se vuelva a calcular. Esto es así porque la posición destino se calcula a partir de la matriz de cámara, y por lo tanto si se mueve la posición del marcador (en la realidad y por lo tanto se ve en cada frame), entonces la matriz de cámara cambia.

6. Si *desplazamiento* no es igual a *actual*, entonces se debe de mover la posición del objeto virtual para ir a *desplazamiento*. Para esto:

    6.1. Se calcula el tiempo transcurrido desde el útimo frame (con *now* y *start*)
    
    6.2. El desplazamiento que necesita hacer el objeto virtual para llegar a *desplazamiento* es *desplazamiento* - *actual*.
    
    6.3. El vector de desplazamiento para llegar se divide entre la norma de este para obtener el vector con longitud 1. Después, se multiplica por el tiempo transcurrido entre 1.5. De este modo, se mueve poco a poco el objeto. Si se obtiene un desplazamiento mayor al que se necesita para llegar al destino, nos quedamos con el desplazamiento total para llegar ignorando lo hecho.
    
    6.4. Se actualiza la posición actual del objeto virtual, sumándole el desplazamiento a realizar (calculado en el punto anterior)
    
    6.5. Se dibuja el objeto virtual en verde en la posición actual. Para esto, se obtienen los píxeles correspondientes a los vértices del objeto (se aplica con htrans tanto la posición actual como la matriz de cámara).
    

7. Si *desplazamiento* es igual a *actual*, se dibuja el objeto en la posición actual. Para esto, se obtienen los píxeles correspondientes a los vértices del objeto (se aplica con htrans tanto la posición actual como la matriz de cámara).

8. Se guarda el tiempo actual en start, de forma que se sepa el tiempo en el que se hizo el último frame.

## Tardanza del código

Para ver cuanto tarda en ejecutar el inicio del código (hasta que se muestra el primer frame), se va a añadir las siguientes líneas de código cuando empieza y cuando se captura el primer frame:

In [ ]:
import time
# ...
inicio = time.time()

# Código que se hace cada frame

fin = time.time()
print(fin-inicio)

Como se puede observar, este código tarda 0 segundos porque no hace nada en medio, pero al añadirlo a `RA.py`, el tiempo aumenta, obteniendo 6.36 segundos de espera antes de obtener el primer frame.

Posteriormente, si se cuenta el tiempo entre frames (de la misma forma), se obtiene:

- Una media de 0.02 segundos si se observa un marcador (varía normalmente entre 0.01 y 0.03, obteniendo en pocas ocasiones 0.04)
- Una media de 0.003 segundos si no se observa ningun contorno en la cámara, y por lo tanto ningún marcador (tiene ocasiones en las que se obtienen muy pocos segundos como 0.001 o más de la media como 0.007)

Aunque se debe esperar un tiempo apreciable antes de comenzar a ver la cámara, gracias al poco tiempo entre frames, una vez se inicia se observa de forma correcta.

## Objetos disponibles

Se puede elegir el objeto con **--objeto**, siendo por defecto una casa (no hace falta indicarlo), pero se puede cambiar por una pirámide (*--objeto "piramide"*) o un reloj de arena (*--objeto "reloj-arena*). Se pueden observar a continuación los objetos:

Casa:

<img src="img/casa.png" style="width:20%"/> </td>

Pirámide:

<img src="img/piramide.png" style="width:20%"/> </td>

Reloj de arena:

<img src="img/reloj-arena.png" style="width:20%"/> </td>

## Prueba

Si se quiere probar el código, pero no se tiene un marcador en físico, se puede probar con la imagen "*marker.jpg*" que se encuentra en esta carpeta, de forma que se ve esta imagen en lugar de la cámara.

Si se quiere hacer, el comando es ***RA.py --dev "./marker.jpg" --loop***, pero también se puede ejecutar con los otros objetos, con ***RA.py --dev "./marker.jpg" --loop --objeto "piramide"*** o ***RA.py --dev "./marker.jpg" --loop --objeto "reloj-arena"***.

Es importante indicar que se ha adaptado el código para la cámara que tenía en casa. Se ha utilizado el código de **calibrate** de los apuntes de la asignatura para obtener la matriz de calibración y almacenarla en la variable **K_Matriz**. Si se quiere usar otra cámara, sería necesario cambiar la matriz de calibración a la usada por esa cámara (cambiar el valor de *K_Matriz*).

## Teoría utilizada

Para realizar el programa, ha sido necesario entender cómo funciona el modelo de cámara (matriz de calibración, matriz de cámara y cómo se transforman los puntos del espacio 3D en píxeles). Todo esto se ha comprendido con las explicaciones de los [apuntes de la asignatura](https://github.com/albertoruiz/umucv/blob/master/notebooks/camera.ipynb).